In [1]:
source_code = bytes("""
                    def print_function(x):
                        print(x)
                   ""","utf-8")

In [3]:
%pip install pyparsing


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pyparsing as pp

# Define the grammar
identifier = pp.Word(pp.alphas + "_", pp.alphas + "_0123456789")
keyword = pp.oneOf("def async class")
func_name = identifier
func_params = pp.nestedExpr("(", ")", content=identifier + pp.Optional("," + identifier)[0, ...])
func_def = pp.Keyword("def") + func_name + func_params + pp.restOfLine
async_func_def = pp.Keyword("async") + func_def
class_def = pp.Keyword("class") + identifier + pp.restOfLine
method_def = func_def | async_func_def

# Define the parser
parser = pp.ZeroOrMore(func_def | async_func_def | class_def)

# Parse a sample Python code
code = """
def add(x, y):
    return x + y

async def async_add(x, y):
    return x + y

class MyClass:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    async def async_method(self, z):
        return self.x + self.y + z
"""

result = parser.parseString(code, parseAll=True)

# Extract information about functions, classes, and methods
functions = []
classes = []
methods = []

for token in result:
    if token[0] == "def":
        func_name = token[1]
        func_params = token[2]
        functions.append({"name": func_name, "params": func_params})
    elif token[0] == "async" and token[1] == "def":
        func_name = token[2]
        func_params = token[3]
        functions.append({"name": func_name, "params": func_params, "async": True})
    elif token[0] == "class":
        class_name = token[1]
        classes.append({"name": class_name})
        for method_token in token[2:]:
            if method_token[0] == "def":
                method_name = method_token[1]
                method_params = method_token[2]
                methods.append({"name": method_name, "params": method_params, "class": class_name})
            elif method_token[0] == "async" and method_token[1] == "def":
                method_name = method_token[2]
                method_params = method_token[3]
                methods.append({"name": method_name, "params": method_params, "class": class_name, "async": True})

print("Functions:")
for func in functions:
    print(f"  {func['name']}({', '.join(func['params'])})")

print("Classes:")
for cls in classes:
    print(f"  {cls['name']}")

print("Methods:")
for method in methods:
    print(f"  {method['name']}({', '.join(method['params'])}) in class {method['class']}")


KeyboardInterrupt: 